In [1]:
import psycopg2
import datetime
import numpy as np
import pandas as pd
from itertools import islice

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from sklearn import preprocessing
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters

from finance.utilities import utils, financial_utils, time_series_utils, cluster_utils, random_forest_utils

In [2]:
register_matplotlib_converters()

%matplotlib notebook
%matplotlib inline

### Stock data

In [3]:
query = """
    select
        e.symbol
        , e.market_datetime
        , e.open
        , e.high
        , e.low
        , e.close
        , e.volume
    from td.equities as e
    where left(e.symbol, 1) = 'A'
    order by e.market_datetime
    limit 100
    """

df = utils.query_db(db_connection=utils.DW_STOCKS_JUPYTER, query=query)
df.head()

,symbol,market_datetime,open,high,low,close,volume
0,AZO,1999-09-20 05:00:00,27.125000,27.87500,27.06250,27.7500,419300.0
1,AMH,1999-09-20 05:00:00,23.625000,23.75000,23.50000,23.5000,3800.0
2,ATR,1999-09-20 05:00:00,12.656250,12.84375,12.25000,12.2500,182200.0
3,AZN,1999-09-20 05:00:00,10.328125,10.37500,10.28125,10.3125,915200.0
4,AHC,1999-09-20 05:00:00,63.437500,63.43750,62.87500,63.1250,324800.0


### Check for gpus

In [4]:
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(1))

True
GeForce GTX 1080 Ti


In [5]:
df = cluster_utils.encode_one_hot(df=df, column='symbol')

In [6]:
for col in ['open', 'high', 'low', 'close', 'volume']:
    df = cluster_utils.normalize(df=df, column=col, subset='symbol')

In [7]:
train = df.copy()
train = train.drop(columns=['symbol'])
train['market_datetime'] = train['market_datetime'].astype(int)

In [8]:
train_target = train['open'].shift(-1)

In [9]:
split_size = 80000
train_x = train.iloc[1:split_size].values
train_y = train_target.iloc[1:split_size].values

test_x = train.iloc[split_size+1:-1].values
test_y = train_target.iloc[split_size+1:-1].values

In [15]:
print(torch.cuda.memory_allocated())

print(torch.cuda.memory_cached())

73216
2097152


In [16]:
tensor_x = torch.tensor(train_x).cuda(0)
tensor_y = torch.tensor(train_y).cuda(0)

In [17]:
input_size = 87
output_size = 1
learning_rate = 0.01 
epochs = 100

model = torch.nn.Linear(input_size, output_size).cuda(0)

criterion = torch.nn.MSELoss().cuda(0)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [18]:
with torch.cuda.device(1):
    for t in range(2):
        prediction = model(tensor_x.float()).cuda(0)

        loss = criterion(prediction, tensor_y.float()).cuda(0)

        optimizer.zero_grad() #.cuda(0)
        loss.backward() #.cuda(0)
        optimizer.step() #.cuda(0)
        
        loss.item()

/home/nautilus/anaconda3/envs/babylon/lib/python3.6/site-packages/torch/nn/modules/loss.py:443: UserWarning: Using a target size (torch.Size([99])) that is different to the input size (torch.Size([99, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/nautilus/anaconda3/envs/babylon/lib/python3.6/site-packages/torch/nn/modules/loss.py:443: UserWarning: Using a target size (torch.Size([99])) that is different to the input size (torch.Size([99, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [14]:
loss.item()

nan